## Regresión lineal con el paquete Scikit learn
Tiene la ventaja de Hacer la selección de variables independientes que mejor modelen la variable dependiente

In [2]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
import pandas as pd

In [3]:
data = pd.read_csv("../Datasets/ads/Advertising.csv")
data.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
## Candidatas a variables independientes
feature_cols = ["Radio","Newspaper","TV"]
X = data[feature_cols]
## Variable dependiente a modelar
Y = data["Sales"]

In [17]:
estimator = SVR(kernel="linear")
selector = RFE(estimator, 2, step=1)
selector = selector.fit(X,Y)

In [18]:
selector.support_

array([ True, False,  True], dtype=bool)

## En este punto hemos determinado las dos variables independientes que mejor modelan las ventas: Radio y TV.
Ahora implementaremos la regresión lineal

In [5]:
from sklearn.linear_model import LinearRegression

In [8]:
X_pred = X[["Radio","TV"]]
Modelo_lineal = LinearRegression()
Modelo_lineal.fit(X_pred,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
Modelo_lineal.coef_

array([ 0.18799423,  0.04575482])

### La ecuación resultante es:
Sales = 0,18799423*Radio + 0,04575482*TV + 2,9210999124

In [10]:
Modelo_lineal.intercept_

2.9210999124051362

### El R2 del modelo es:

In [12]:
Modelo_lineal.score(X_pred,Y)

0.89719426108289557